In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from types import SimpleNamespace
import numpy as np
import time
import commentjson
from rich import print
import cv2
from collections import OrderedDict

from yolact_pkg.utils.augmentations import SSDAugmentation, BaseTransform
from yolact_pkg.data.config import Config
from yolact_pkg.yolact import Yolact
from yolact_pkg.eval import infer, annotate_img, evaluate, parse_args, calc_map, calc_map_classwise
from yolact_pkg.data.config import MEANS
from yolact_pkg.train import train
from yolact_pkg.data.coco import COCODetection, detection_collate
import torch

In [ ]:
# print("dir()", dir())
torch.cuda.empty_cache()
    
    
dataset = Config({
    'name': 'Base Dataset',

    # Training images and annotations
    'train_images': '/home/sruiz/datasets2/reconcycle/2022-05-02_kalo_qundis/coco',
    'train_info':   '/home/sruiz/datasets2/reconcycle/2022-05-02_kalo_qundis/coco/_train.json',

    # Validation images and annotations.
    'valid_images': '/home/sruiz/datasets2/reconcycle/2022-05-02_kalo_qundis/coco',
    'valid_info':   '/home/sruiz/datasets2/reconcycle/2022-05-02_kalo_qundis/coco/_test.json',

    # Whether or not to load GT. If this is False, eval.py quantitative evaluation won't work.
    'has_gt': True,

    # A list of names for each of you classes.
    'class_names': ("hca_front", "hca_back", "hca_side1", "hca_side2", "battery", "pcb", "internals", "pcb_covered", "plastic_clip"),

    # COCO class ids aren't sequential, so this is a bandage fix. If your ids aren't sequential,
    # provide a map from category_id -> index in class_names + 1 (the +1 is there because it's 1-indexed).
    # If not specified, this just assumes category ids start at 1 and increase sequentially.
    'label_map': None
})

config_override = {
    'name': 'yolact_base',

    # Dataset stuff
    'dataset': dataset,
    'num_classes': len(dataset.class_names) + 1,

    # Image Size
    'max_size': 1100, #! I changed this, was 550
    
    'save_path': 'data_full/yolact/2022-10-17_kalo_qundis/',
    
    # we can override args used in eval.py:
    'score_threshold': 0.1,
    'top_k': 10
}

# we can override training args here:
training_args_override = {
    "batch_size": 2, #! I changed this, was 8
    "save_interval": -1, # -1 for saving only at end of the epoch
    # "resume": 
    "validation_size": 100,
}
print("test")
yolact = Yolact(config_override)

In [ ]:
###########################################
# Inference                               #
###########################################

yolact.eval()
yolact.load_weights("./data_limited/yolact/2022-05-02_kalo_qundis/yolact_base_274_202125.pth")

# frame, classes, scores, boxes, masks = yolact.infer("/root/datasets/2022-05-02_kalo_qundis/coco/JPEGImages/1066.jpg")

# annotated_img = annotate_img(frame, classes, scores, boxes, masks)

# cv2.namedWindow("output", cv2.WINDOW_NORMAL)
# cv2.imshow("output",annotated_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# 
# todo: evaluation

val_dataset = COCODetection(image_path=yolact.cfg.dataset.valid_images,
                            info_file=yolact.cfg.dataset.valid_info,
                            transform=BaseTransform(MEANS))

args = parse_args()

# print("args", args)
all_maps, ap_data = evaluate(yolact, val_dataset)


In [ ]:
print(len(yolact.cfg.dataset.class_names), yolact.cfg.dataset.class_names)

iou_thresholds = [x / 100 for x in range(50, 100, 5)]

# all_maps2, _ = calc_map_classwise(yolact, ap_data)

# print("all_maps2", all_maps2)

def calc_map_classwise_copy(net:Yolact, ap_data):
    print('Calculating mAP...')
    aps = [{'box': [], 'mask': []} for _ in iou_thresholds]

    all_class_maps = []
    for _class in range(len(net.cfg.dataset.class_names)):
        
        for iou_idx in range(len(iou_thresholds)):
            for iou_type in ('box', 'mask'):
                ap_obj = ap_data[iou_type][iou_idx][_class]

                if not ap_obj.is_empty():
                    aps[iou_idx][iou_type].append(ap_obj.get_ap())

        class_map = {'box': OrderedDict(), 'mask': OrderedDict()}

        # Looking back at it, this code is really hard to read :/
        for iou_type in ('box', 'mask'):
            class_map[iou_type]['all'] = 0 # Make this first in the ordereddict
            for i, threshold in enumerate(iou_thresholds):
                mAP = sum(aps[i][iou_type]) / len(aps[i][iou_type]) * 100 if len(aps[i][iou_type]) > 0 else 0
                class_map[iou_type][int(threshold*100)] = mAP
            class_map[iou_type]['all'] = (sum(class_map[iou_type].values()) / (len(class_map[iou_type].values())-1))
        
        # pretty results:
        # print("class:", net.cfg.dataset.class_names[_class])
        # print_maps(class_map)
        
        # Put in a prettier format so we can serialize it to json during training
        class_map = {k: {j: round(u, 2) for j, u in v.items()} for k, v in class_map.items()}
        all_class_maps.append(class_map)
    
    # work out averages for rotation difference
    for _class in range(len(net.cfg.dataset.class_names)):
        iou_idx = 0
        iou_type = 'rot'
        
        ap_obj = ap_data[iou_type][iou_idx][_class]
        print("class:", net.cfg.dataset.class_names[_class])

        rot_diffs = [i[0] for i in ap_obj.data_points]
        print("rot_diffs", rot_diffs)
        
        # ! sometimes the numbers are 0.0, 90.0, 180.0
        # ! obviously something is wrong somewhere because that is not correct.

        print("avg", np.mean(rot_diffs))
        print("max", np.max(rot_diffs))


    
    return all_class_maps, ap_data

all_maps2, _ = calc_map_classwise_copy(yolact, ap_data)


In [ ]:
# todo: compute the rotation accuracy




In [ ]:
# todo: compute location accuracy in real world coordinates